# The Battle of Neighborhoods

### Import libraries

In [1]:
#Import pandas
import pandas as pd

### Read data from url

In [2]:
#Url with Uruguay postal codes 
url = 'https://www.correo.com.uy/IsisBusquedaDireccionPlugin/localidades.jsp'
#Store Data Frame
df_uy = pd.read_excel(url,header=1)
#Print message
print('Data stored')

Data stored


### Modify dataframe

In [3]:
#Rename columns
headers =['Department', 'Neighborhood', 'PostalCode']
df_uy.columns = headers
#Change first character of each word to uppercase and remaining to lowercase in Borough and Neighborhood cells
df_uy['Department'] = df_uy['Department'].str.title()
df_uy['Neighborhood'] = df_uy['Neighborhood'].str.title()
#Change columns order in data frame
df_uy = df_uy[["PostalCode", "Department", "Neighborhood"]]

### Print dimensions and search for null values

In [4]:
#Print number of rows
print('Number of rows: '+str(df_uy.shape[0]))
#Print number of columns
print('Number of columns: '+str(df_uy.shape[1]))
#Print number of null values
print('Null values in dataframe:')
print(str(df_uy.isnull().sum()))

Number of rows: 2061
Number of columns: 3
Null values in dataframe:
PostalCode      0
Department      0
Neighborhood    0
dtype: int64


### Create a dataframe with Montevideo postal codes and neighborhoods

In [5]:
#Save Montevideo postal codes and neighborhoods in df_mvd
df_mvd = df_uy.loc[df_uy['Department']=='Montevideo',['PostalCode','Neighborhood']]
#Combined rows with equal "PostalCode" values and apply function to each 'Neighborhood' cell  
df_mvd.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
#Restar index values
df_mvd.reset_index(drop=True,inplace=True)
#Print number of rows
print('Number of rows: '+str(df_mvd.shape))

Number of rows: (113, 2)


### Import libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Store coordinates of each location in Montevideo

In [7]:
#Create 'Latitude' and 'Longitude 'columns
df_mvd['Latitude']=None
df_mvd['Longitude']=None

#Iterate over df_mvd with index value of each row as the iterator
for i in df_mvd.index:
    address = str(df_mvd.loc[i,'PostalCode'])+',Montevideo,UY'
    geolocator = Nominatim(timeout=3)
    location = geolocator.geocode(address)
#Store coordinates of each location
    df_mvd.loc[i,'Latitude']=location.latitude
    df_mvd.loc[i,'Longitude']=location.longitude
print('Coordinates stored')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


Coordinates stored


### Foursquare credentials

In [8]:
CLIENT_ID = 'WJSLZFMYFZZIMASD3SDPXRCC3XVNEXSA3XNA2CLN33TQBP3Z' # your Foursquare ID
CLIENT_SECRET = 'QHTE3T14S1GUL45YHSAKRU4JS2AFODNPWAZGVCQYFVXYE3ZH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WJSLZFMYFZZIMASD3SDPXRCC3XVNEXSA3XNA2CLN33TQBP3Z
CLIENT_SECRET:QHTE3T14S1GUL45YHSAKRU4JS2AFODNPWAZGVCQYFVXYE3ZH


### Create function to explore neighborhoods

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
mvd_venues = getNearbyVenues(names=df_mvd['Neighborhood'],
                                 latitudes=df_mvd['Latitude'],
                                 longitudes=df_mvd['Longitude'])

Abayuba


KeyError: 'groups'

### Check the size of the resulting dataframe

In [ ]:
#Check the size of the resulting dataframe
print(mvd_venues.shape)
mvd_venues.head()

### Check number of venues by category

In [ ]:
mvd_venues['Venue Category'].value_counts()

### Check how many venues were returned for each neighborhood

In [ ]:
#Check how many venues were returned for each neighborhood
mvd_venues.groupby('Neighborhood').count()

### Check how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(mvd_venues['Venue Category'].unique())))

### Analyze Each Neighborhood

In [ ]:
# one hot encoding
mvd_onehot = pd.get_dummies(mvd_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mvd_onehot['Neighborhood'] = mvd_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mvd_onehot.columns[-1]] + mvd_onehot.columns[:-1].tolist()
mvd_onehot = mvd_onehot[fixed_columns]

mvd_onehot.head()

### Examine the new dataframe size.

In [ ]:
mvd_onehot.shape

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
mvd_grouped = mvd_onehot.groupby('Neighborhood').mean().reset_index()
mvd_grouped

In [ ]:
#Check new size
mvd_grouped.shape

### Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in mvd_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mvd_grouped[mvd_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Put into a *pandas* dataframe

In [ ]:
#Create a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#Create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mvd_grouped['Neighborhood']

for ind in np.arange(mvd_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mvd_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Cluster Neighborhoods

In [ ]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

mvd_grouped_clustering = mvd_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mvd_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
#Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mvd_merged = df_mvd

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mvd_merged = mvd_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mvd_merged.head() # check the last columns!

### Visualize the resulting clusters

In [ ]:
#In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent to_explorer, as shown below.

address = 'Montevideo, Uruguay'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montevideo, Uruguay are {}, {}.'.format(latitude, longitude))

### Create Map

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mvd_merged['Latitude'], mvd_merged['Longitude'], mvd_merged['Neighborhood'], mvd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [ ]:
#Cluster 1
mvd_merged.loc[mvd_merged['Cluster Labels'] == 0, mvd_merged.columns[[1] + list(range(5, mvd_merged.shape[1]))]]

In [ ]:
#Cluster 2
mvd_merged.loc[mvd_merged['Cluster Labels'] == 1, mvd_merged.columns[[1] + list(range(5, mvd_merged.shape[1]))]]

In [ ]:
#Cluster 3
mvd_merged.loc[mvd_merged['Cluster Labels'] == 2, mvd_merged.columns[[1] + list(range(5, mvd_merged.shape[1]))]]

In [ ]:
#Cluster 4
mvd_merged.loc[mvd_merged['Cluster Labels'] == 3, mvd_merged.columns[[1] + list(range(5, mvd_merged.shape[1]))]]

In [ ]:
#Cluster 5
mvd_merged.loc[mvd_merged['Cluster Labels'] == 4, mvd_merged.columns[[1] + list(range(5, mvd_merged.shape[1]))]]